# Computing the mean

In this notebook, we compute the mean of an array of random numbers.  

Rather than import NumPy, we will use the `%pylab` magic command, which essentially is equivalent

    from numpy import *
    from scipy import *
    from matplotlib.pyplot import *

In [ ]:
%pylab
import time, multiprocessing

In [ ]:
N = 2**28
x = random.rand(N)

In [ ]:
def vsum(x,q):
    q.put(sum(x))
    
def compute_mean(x,np):
    N = len(x)
    jobs = []
    q = multiprocessing.Queue()
    for i in range(np):
        i1 = int(i*N/np)
        i2 = int((i+1)*N/np)
        job = multiprocessing.Process(target=vsum,args=[x[i1:i2],q])
        jobs.append(job)
        
    for job in jobs:
        job.start()
    
    for job in jobs:
        job.join()

    s = 0
    while not q.empty():
        s += q.get()
        
    return s/N    

In [ ]:
# Use 'timeit' loop (see 'using_timeit' notebook.)
procs = 2**arange(0,6)
tdata = empty(procs.shape)
for i,np in enumerate(procs):
    tr = %timeit -o compute_mean(x,np)
    tdata[i] = tr.best    

In [ ]:
%matplotlib notebook
import pandas
p = pandas.Series(procs,dtype='d')
t = pandas.Series(tdata,dtype='d')
df = pandas.DataFrame({'p' : p, 't' : t})
display(df)

In [ ]:
df.plot(x='p', logx=True, logy=True, style='.-',markersize=20)